In [1]:
import os
import json
import pandas as pd
import traceback


In [3]:
from langchain_google_genai import GoogleGenerativeAI

In [ ]:
from dotenv import load_dotenv
load_dotenv()


True

In [12]:
import os 
key=os.getenv("GOOGLE_API_KEY")


In [10]:
llm= GoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.2,
    max_output_tokens=1024,
    top_p=0.8,
    top_k=40,
    api_key=key
)

In [11]:
llm

GoogleGenerativeAI(model='gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.2, top_p=0.8, top_k=40, max_output_tokens=1024, client=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.2, top_p=0.8, top_k=40, max_output_tokens=1024, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000016EE1CB5BB0>, default_metadata=(), model_kwargs={}))

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
prompt = PromptTemplate(
    input_variables=["input"],
    template="You are a helpful assistant. Answer the question: {input}"
)